In [1]:
from pandas import set_option as pd_set_option
pd_set_option('display.max_rows', 10000)
pd_set_option('display.max_columns', 10000)
from excel_app import getDataFromExcel
from pandas import concat as pd_concat
from pandas import merge as pd_merge
from pandas import MultiIndex as pd_MultiIndex
from excel_app import df_to_excel
from reports.contracts.contract import  get_xls_struct

def get_contract_report():
    xls_struct = get_xls_struct()
    df_period = getDataFromExcel("files//periods.xlsx", list(xls_struct[0]))
    df_period = df_period.rename(columns=xls_struct[0])
    df_rev = getDataFromExcel("files//revenew.xlsx", list(xls_struct[1]))
    df_rev = df_rev.rename(columns=xls_struct[1])
#     df_rev = df_rev.loc[df_rev['rev_treat'] == '"Договор 00000000052 от 25.01.2012 13:03:37"']
    
#     df_exp = getDataFromExcel("files//expense.xlsx", list(xls_struct[2]))
#     df_exp = df_exp.rename(columns=xls_struct[2])

#     """union and merge table"""
#     df_result = pd_concat([df_rev, df_exp], ignore_index=True, sort=False)
#     df_result = df_result.loc[df_result['rev_treat'] == '"Договор 00000000052 от 25.01.2012 13:03:37"']
#     df_result = pd_merge(df_result, df_period, on=["treat", "branch"], how="left")
#     df_result = df_result.fillna(
#         {"date": "01.01.2019 0:00:00", "pay_plan": 0, "pay_fact": 0, "revenue_plan": 0, "revenue_fact": 0}).fillna(
#         '')
#     """building pivot table"""
#     df_result = df_result.pivot_table(index=xls_struct[3], columns="date",
#                                       values=["pay_plan", "pay_fact", "revenue_plan", "revenue_fact"], fill_value='',
#                                       margins=True,
#                                       aggfunc=sum).replace(0, '')
#     """removing the last row with totals"""
#     df_result = df_result[:-1]
#     """to swipe columns"""
#     # result = result.swaplevel(0, 1, axis=1).sort_index(axis=1)
#     """reseting and building a new hierarchy indexes"""
#     df_result = df_result.reset_index()
#     df_result = df_result.set_index(['rev_treat', 'exp_treat']).sort_index()

#     return df_result
    return df_rev

get_contract_report()

,branch,treat,rev_treat,treat_client,treat_inn,treat_num,treat_date,treat_end_date,treat_object,treat_type,treat_amount,amount_done_on_begin,amount_on_begin,pay_on_begin,amount_done_on_end,amount_on_end
0,Центральный офис (ЦО),"""Договор 00000006194 от 15.07.2019 0:00:00""","""Договор 00000006194 от 15.07.2019 0:00:00""","""ИнжЭнергоПроект АО""","""7725592822""",""" Договор уступки прав требования (цессии)""",15.07.2019 0:00:00,01.01.0001 0:00:00,"""Без объекта""","""Договор уступки права требования долга (цессия)""",1.336413e+06,,1.336413e+06,,,1.336413e+06
1,Центральный офис (ЦО),"""Договор 00000006205 от 19.07.2019 0:00:00""","""Договор 00000006205 от 19.07.2019 0:00:00""","""Мэйджор Кар Плюс ООО""","""5024079859""","""19919""",19.07.2019 0:00:00,01.01.0001 0:00:00,"""Без объекта""","""Договор купли-продажи""",4.666667e+04,,4.666667e+04,,,4.666667e+04
2,Центральный офис (ЦО),"""Договор 00000004793 от 01.12.2019 0:00:00""","""Договор 00000004793 от 01.12.2019 0:00:00""","""Моисеев Денис Александрович ИП""","""771608808196""","""АНП-12/2019-Моисеев""",01.12.2019 0:00:00,31.10.2020 0:00:00,"""Без объекта""","""Договор аренды""",0.000000e+00,,0.000000e+00,,,0.000000e+00
3,Центральный офис (ЦО),"""Договор 00000001733 от 27.03.2013 12:00:02""","""Договор 00000001733 от 27.03.2013 12:00:02""","""ЭСК СОЮЗ АО""","""7728620323""","""0896-ЭСК""",27.03.2013 12:00:02,20.01.2015 0:00:00,"""Дягилевская ТЭЦ""","""Договор строительного подряда на основе генпо...",1.694915e+08,8.95491e+06,1.605366e+08,1e+08,8.95491e+06,1.605366e+08
4,Центральный офис (ЦО),"""Договор 00000005413 от 19.12.2017 0:00:00""","""Договор 00000005413 от 19.12.2017 0:00:00""","""Объединенная энергетическая компания АО""","""7720522853""","""17913-КС-17""",19.12.2017 0:00:00,31.12.2018 0:00:00,"""ПКЛ 20 кВ № 34-1""","""Договор строительного подряда на основе генпо...",7.254508e+07,,7.254508e+07,,,7.254508e+07
5,Центральный офис (ЦО),"""Договор 00000005415 от 19.12.2017 0:00:00""","""Договор 00000005415 от 19.12.2017 0:00:00""","""Объединенная энергетическая компания АО""","""7720522853""","""18199-КС-17""",19.12.2017 0:00:00,31.12.2018 0:00:00,"""ПКЛ 20 кВ № 35-1""","""Договор строительного подряда на основе генпо...",2.179551e+08,,2.179551e+08,,,2.179551e+08
6,Центральный офис (ЦО),"""Договор 00000005411 от 18.12.2017 0:00:00""","""Договор 00000005411 от 18.12.2017 0:00:00""","""Объединенная энергетическая компания АО""","""7720522853""","""17750-КС-17""",18.12.2017 0:00:00,31.12.2018 0:00:00,"""ПКЛ 20 кВ № 27-1 ""","""Договор строительного подряда на основе генпо...",5.510835e+06,4.51512e+06,9.957159e+05,4.8146e+06,4.51512e+06,9.957159e+05
7,Центральный офис (ЦО),"""Договор 00000006058 от 12.02.2019 0:00:00""","""Договор 00000006058 от 12.02.2019 0:00:00""","""Мэйджор Кар Плюс ООО""","""5024079859""","""3751""",12.02.2019 0:00:00,01.01.0001 0:00:00,"""Без объекта""","""Договор купли-продажи""",2.816667e+05,,2.816667e+05,,,2.816667e+05
8,Центральный офис (ЦО),"""Договор 00000005051 от 13.06.2016 0:00:00""","""Договор 00000005051 от 13.06.2016 0:00:00""","""МОЭК ПАО""","""7720518494""","""24-СМР-МТ/16""",13.06.2016 0:00:00,17.12.2016 0:00:00,"""Присоединение к тепловым сетям ПАО МОЭК""","""Договор строительного подряда на основе генпо...",2.915227e+06,,2.915227e+06,,1.51471e+06,1.400516e+06
9,Центральный офис (ЦО),"""Договор 00000006076 от 25.04.2019 0:00:00""","""Договор 00000006076 от 25.04.2019 0:00:00""","""ТЭР ООО Московский филиал ""","""7733667768""","""АТС/2019-ТЭР""",25.04.2019 0:00:00,31.10.2019 0:00:00,"""Без объекта""","""Договор аренды транспортного средства без эки...",2.942373e+05,,2.942373e+05,,,2.942373e+05
